In [1]:
import pandas as pd
import sqlite3

# Шаг 1 - Чтение публичного CSV файла в pandas

Шаг 1
(Балл - 0.2) Необходимо:
- скачать CSV-файл — «final_transactions.csv»,
- создать таблицу transaction_bd со всеми полями,
- загрузить данные из файла в таблицу и оставить таблицу со структурой:

      (0, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - время транзакций
      (1, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - клиент
      (2, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакций

Комментарий: Можно оставить все 5 полей, но запросы будут выполняться чуть дольше.

In [2]:
# публичная ссылка на файл 'final_transactions.csv'
url_file = 'https://drive.google.com/file/d/1_YrZPaD1zNCcXUkUfeWM0ZOdg_was_iA/view'

# замена левой части адреса
main = 'https://drive.google.com/uc?id='

# Берем id файла 
id_url = url_file.split('/')[-2]

# url пригодный для чтения
direct_url = main + id_url

# cкачиваем и читаем файл
df = pd.read_csv(direct_url)
df.head()


,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66


## Создание пустой базы

In [3]:
from sqlite3 import Error 
def create_connection(path):
    '''
    Подключаемся к имеющейся или создаем новую
    есл не найдена
    '''
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection


def execute_query(connection, query):
    """
    Передаем запрос в бакзу с созданным 
    ранее connection
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [4]:
# создаем базу transaction_bd
data_base = 'Transaction_bd.db'
connection = create_connection(data_base)

Connection to SQLite DB successful


## Создание таблицы в базе

In [5]:
df.columns

Index(['TRANSACTION_ID', 'TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID',
       'TX_AMOUNT'],
      dtype='object')

In [6]:
# создаем запрос на таблицу пустую
table = 'Transactions'
create_table = f"""
CREATE TABLE IF NOT EXISTS {table} (
  TRANSACTION_ID INTEGER PRIMARY KEY AUTOINCREMENT,
  TX_DATETIME NUMERIC NOT NULL,
  CUSTOMER_ID INTEGER,
  TERMINAL_I INTEGER,
  TX_AMOUNT REAL
);
"""

In [7]:
# создаем в базе таблицу  Transactions
execute_query(connection, create_table)

Query executed successfully


## Наполнение таблицы  из датафрейма

In [8]:
# создаем в базе таблицу  Transactions из датафрейма
df.to_sql(table, connection, if_exists='replace', index=False)

1048575

In [9]:
# смотрим базу
sql_string = 'SELECT * FROM Transactions'
df_check = pd.read_sql(sql_string, connection,  index_col='TRANSACTION_ID')
df_check.tail()

,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
TRANSACTION_ID,,,,
1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,2023-04-20 10:07:28,738,5151,20.38
1048572,2023-04-20 10:07:33,1000,5417,182.79
1048573,2023-04-20 10:07:39,3028,6439,455.44
1048574,2023-04-20 10:07:43,3252,6009,304.55


## Удаление лишних колонок из таблицы базы SQL

In [10]:
DROP_COLUMNS = ('TERMINAL_ID',) # TRANSACTION_ID
# удаляем столбцы лишние по заданию
for column in DROP_COLUMNS:  
    cursor = connection.cursor()
    alter_column = f'ALTER TABLE {table} DROP COLUMN {column}'
    cursor.execute(alter_column)
    connection.commit()

In [11]:
# смотрим базу
sql_string = 'SELECT * FROM Transactions'
df_check = pd.read_sql(sql_string, connection, index_col='TRANSACTION_ID')
df_check.head()

,TX_DATETIME,CUSTOMER_ID,TX_AMOUNT
TRANSACTION_ID,,,
0,2023-01-01 00:00:31,596,533.07
1,2023-01-01 00:02:10,4961,808.56
2,2023-01-01 00:07:56,2,1442.94
3,2023-01-01 00:09:29,4128,620.65
4,2023-01-01 00:10:34,927,490.66


# Шаг 2
(Балл - 0.1 за каждый пункт) Написать следующие запросы к таблице transaction_bd:

- A. Вывести всех клиентов, у которых сумма транзакций больше 700000 за весь период (сортируя клиентов по возрастанию);
- B. Вывести всех клиентов, у которых сумма транзакций больше 200000 за период 01.01.2023 - 13.01.2023 (сортируя клиентов по возрастанию);
- C. Вывести тех клиентов, у которых id начинается с 4 и количество транзакций за весь период более 444;
- D. Создать флаг доходности клиентов по логике:
    1. Если сумма транзакций не более 50000, тогда вывести 'низкая доходность';
    2. Если сумма транзакций больше 50000 и не более 10000 тогда вывести 'средняя доходность';
    3. Если сумма транзакций больше 100000 тогда вывести 'высокая доходность'.
- E. Посмотреть количество клиентов с каждым видом доходности (из пункта 2.d);
- F. Вывести сумму транзакций за каждый день (сортируя дни по возрастанию).


Оконные функции SQL простым языком с примерами - https://habr.com/ru/post/664000/

### A. Вывести всех клиентов, у которых сумма транзакций больше 700000 за весь период (сортируя клиентов по возрастанию);

In [12]:
limit = 700000
query_a = f'''
SELECT CUSTOMER_ID
FROM {table}
GROUP BY CUSTOMER_ID
HAVING sum(TX_AMOUNT) > {limit}
ORDER BY CUSTOMER_ID ASC
'''
# получаем датафрейм по запросу
data = pd.read_sql(query_a, connection)
data

,CUSTOMER_ID
0,389
1,2249
2,2891
3,3116
4,4163


### B. Вывести всех клиентов, у которых сумма транзакций больше 200000 за период 01.01.2023 - 13.01.2023 (сортируя клиентов по возрастанию);

In [13]:
limit = 200000
query_b = f'''
SELECT CUSTOMER_ID
FROM {table}
GROUP BY CUSTOMER_ID
HAVING sum(TX_AMOUNT) > {limit} AND
TX_DATETIME BETWEEN '2023-01-01' AND '2023-01-13'
ORDER BY CUSTOMER_ID ASC 
'''
# получаем датафрейм по запросу
data = pd.read_sql(query_b, connection)
data

,CUSTOMER_ID
0,1
1,4
2,9
3,25
4,27
...,...
947,4974
948,4981
949,4983
950,4994


### C. Вывести тех клиентов, у которых id начинается с 4 и количество транзакций за весь период более 444;

In [14]:
limit = 444
query_c = f'''
SELECT CUSTOMER_ID
FROM {table}
where SUBSTRING(CUSTOMER_ID, 1, 1) = '4'
GROUP BY CUSTOMER_ID
HAVING sum(TX_AMOUNT) > {limit}
ORDER BY CUSTOMER_ID ASC 
'''
# получаем датафрейм по запросу
data = pd.read_sql(query_c, connection)
data

,CUSTOMER_ID
0,4
1,40
2,41
3,42
4,43
...,...
1095,4995
1096,4996
1097,4997
1098,4998


### D. Создать флаг доходности клиентов по логике:
    1. Если сумма транзакций не более 50000, тогда вывести 'низкая доходность';
    2. Если сумма транзакций больше 50000 и не более 100000 тогда вывести 'средняя доходность';
    3. Если сумма транзакций больше 100000 тогда вывести 'высокая доходность'.

In [15]:
level_1 = 50000
level_2 = 100000
query_d = f'''
SELECT CUSTOMER_ID,
CASE WHEN sum(TX_AMOUNT) <= {level_1} THEN 'низкая доходность'
     WHEN sum(TX_AMOUNT) > {level_1} and sum(TX_AMOUNT) <= {level_2} THEN 'средняя доходность'
ELSE 'высокая доходность' 
END  RATE
FROM {table}
GROUP BY CUSTOMER_ID
ORDER BY CUSTOMER_ID ASC 

'''
# получаем датафрейм по запросу
data = pd.read_sql(query_d, connection)
data

,CUSTOMER_ID,RATE
0,0,высокая доходность
1,1,высокая доходность
2,2,высокая доходность
3,3,низкая доходность
4,4,высокая доходность
...,...,...
4981,4995,высокая доходность
4982,4996,низкая доходность
4983,4997,высокая доходность
4984,4998,высокая доходность


### E. Посмотреть количество клиентов с каждым видом доходности (из пункта 2.d);

In [16]:
# берем предыдущий запрос и вставляем его в новый
query_e = f'''
WITH TAB as ({query_d})
SELECT TAB.RATE, count(TAB.RATE) as QTY
FROM TAB
GROUP BY RATE
'''
# получаем датафрейм по запросу
data = pd.read_sql(query_e, connection)
data

,RATE,QTY
0,высокая доходность,2167
1,низкая доходность,1812
2,средняя доходность,1007


### F. Вывести сумму транзакций за каждый день (сортируя дни по возрастанию).

In [22]:
# берем предыдущий запрос и вставляем его в новый
query_f = f'''
SELECT SUBSTRING(TX_DATETIME, 1, 10) as DATE,
       sum(TX_AMOUNT) as SUMM
FROM  {table}
GROUP BY DATE
ORDER BY DATE ASC
'''
# получаем датафрейм по запросу
data_dash = pd.read_sql(query_f, connection)
data_dash

,DATE,SUMM
0,2023-01-01,4827656.26
1,2023-01-02,4862551.41
2,2023-01-03,5058973.71
3,2023-01-04,4938142.47
4,2023-01-05,5002954.23
...,...,...
105,2023-04-16,5299386.81
106,2023-04-17,5100973.31
107,2023-04-18,5233557.39
108,2023-04-19,5194846.44


# Шаг 3
(Балл - 0.2) 

- Подготовить дашборд с помощью Dash по пункту 2.f, включив туда графики bar и histogram;
- Bставить в конце ноутбука скрин графиков из дашборда.


## Подготовка дашборда

In [18]:
#!pip install dash
#!pip install jupyter-dash

In [19]:
from dash import Dash, html, dcc
import plotly.express as px

## Графики

Шпаргалка по визуализации данных в Python с помощью Plotly - https://habr.com/ru/post/502958/

In [24]:
app = Dash()
title_1 = 'Сумма транзакций за каждый день'
title_2 = 'Распределение сумм транзакций в день'

fig1 = px.bar(data_dash, x="DATE", y="SUMM", title = title_1)

fig2 = px.histogram(data_dash, x = "SUMM",
             title = title_2,
             nbins = 100,
             )
fig1.update_layout(template = 'plotly_dark') # темная тема
fig2.update_layout(template = 'plotly_dark') # темная тема

# публикация на веб_сервере для браузера
app.layout = html.Div(children=[
    html.H1(children = title_1),

    html.Div(children = title_2),

    dcc.Graph(
        id='example-graph',
        figure=fig1
    ),
     dcc.Graph(
        id='example-graph1',
        figure=fig2
    )
])

app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [29/Mar/2023 20:56:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 20:56:22] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_1m1679933151.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 20:56:22] "GET /_dash-component-suites/dash/deps/react@16.v2_9_1m1679933151.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 20:56:22] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_9_1m1679933151.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 20:56:22] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_9_1m1679933151.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 20:56:22] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_1m1679933151.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 20:56:22] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_9_0m1679933151.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2023 20:56:22] "GET /_dash-component-suites/dash

<img src="https://raw.githubusercontent.com/Mike030668/MIPT_magistratura/main/Data%20storage%20and%20processing%20systems/images/img-2023-03-29-20-57-12.png" alt="Screenshort" />